In [6]:
#imports
import pandas as pd
import numpy as np
from IPython.display import Markdown as md
import matplotlib.pyplot as plt

In [152]:
placeholder = './'

In [153]:
# Load data 
ingredients = pd.read_csv(placeholder + 'ingredients.csv')
ingredients['IngredientName'] = ingredients['IngredientName'].str.strip() #stripping whitespace
ingredients.head()

,IngredientName,IngredientShortDescription,IngredientId,PortionUOMTypeId
0,Bread,Bread Stick 12 inch/30cm,1,11.0
1,Bacon,Bacon,2,11.0
2,Chicken Strips,Chicken Strips,3,15.0
3,Chicken. oven roasted patty,"Chicken, single piece",4,15.0
4,Cold Cut Combo,Cold Cut Combo,5,15.0


In [154]:
ingredients.loc[ingredients['IngredientName'] == 'Lettuce']

,IngredientName,IngredientShortDescription,IngredientId,PortionUOMTypeId
26,Lettuce,Lettuce,27,15.0


In [155]:
# Load data 
uom = pd.read_csv(placeholder + 'portion_uom_types.csv')
uom.head(10)

,PortionTypeDescription,PortionUOMTypeId
0,Each,11
1,Fluid Ounce,12
2,Gram,13
3,Milliliter,14
4,Ounce,15


#### From the above 2 tables, we can infer that Lettuce is measured in ounce and it's ingredient id is 27.

In [156]:
#recipe_ingredient_assignments table
recp_ing = pd.read_csv(placeholder + 'recipe_ingredient_assignments.csv')
recp_ing.head(10)

,RecipeId,IngredientId,Quantity
0,2,7,4
1,6,7,4
2,7,7,2
3,8,7,2
4,9,7,2
5,10,13,4
6,11,13,4
7,12,13,2
8,13,13,2
9,14,13,2


In [157]:
lettuce_recipes = recp_ing.loc[recp_ing['IngredientId'] == 27]
#lettuce_recipes.drop(lettuce_recipes[lettuce_recipes['Quantity'] == 0].index, inplace = True)
lettuce_recipes = lettuce_recipes.loc[lettuce_recipes["Quantity"] > 0 ]
lettuce_recipes.shape

(30, 3)

In [158]:
# checking for duplicate recipe ids in the lettuce recipe dataframe
duplicate = lettuce_recipes[lettuce_recipes.duplicated('RecipeId')]
duplicate

,RecipeId,IngredientId,Quantity


In [159]:
# total number of recipes using lettuce
lettuce_recipes_count = len(lettuce_recipes)
md("Total number of recipes using lettuce are : {} .".format(lettuce_recipes_count))

Total number of recipes using lettuce are : 30 .

In [4]:
#lettuce_recipes

##  lettuce_sub_recipes - (6 recipes) 
#### (These are the sub_recipes that have lettuce in them)

In [161]:
#sub-recipe_ingredient_assignments table
sub_recp_ing = pd.read_csv(placeholder + 'sub_recipe_ingr_assignments.csv')
sub_recp_ing.head(10)

,SubRecipeId,IngredientId,Quantity
0,1,1,1
1,1,135,2
2,1,138,1
3,1,146,1
4,1,157,1
5,3,16,1
6,3,135,2
7,3,138,1
8,3,146,1
9,3,157,1


In [162]:
lettuce_sub_recipes = sub_recp_ing.loc[sub_recp_ing['IngredientId'] == 27]

In [163]:
lettuce_sub_recipes = lettuce_sub_recipes.loc[lettuce_sub_recipes["Quantity"] > 0 ]
lettuce_sub_recipes

,SubRecipeId,IngredientId,Quantity
15,5,27,4
26,10,27,2
239,49,27,2
253,51,27,2
279,58,27,4
329,73,27,2


In [164]:
# total number of sub recipes using lettuce
sub_recp_count = len(lettuce_sub_recipes)
md("Total number of SUB recipes using lettuce are: {} ".format(sub_recp_count))

Total number of SUB recipes using lettuce are: 6 

## Recipe-Subrecipe assignement (recipes using sub_recipe)

### convert lettuce_sub_recipes into a list

In [165]:
letSubRecipes = lettuce_sub_recipes['SubRecipeId'].tolist()
letSubRecipes

[5, 10, 49, 51, 58, 73]

In [166]:
## Recipes using sub recipes with lettuce
recp_sub_recp_ass = pd.read_csv(placeholder + 'recipe_sub_recipe_assignments.csv')
recp_sub_recp_ass.head()

,RecipeId,SubRecipeId,Factor
0,2,3,1
1,2,9,1
2,2,10,1
3,2,12,1
4,6,1,1


###  recipes that are using subrecipes that has lettuce

In [167]:
# list of sub-recipes that has lettuce
rec_using_let_srec = recp_sub_recp_ass[recp_sub_recp_ass['SubRecipeId'].isin(letSubRecipes)]

# list of recipes using subrecipes that has lettuce
pd.set_option('display.max_rows', None)
#rec_using_let_srec

In [168]:
rec_using_let_srec.shape


(2478, 3)

### total number of recipe-subrecipe mapping with factor = 0, this doesn't give any info about how much sub-recipe to use, let's drop them

In [169]:
# list recipe - sub recipe assignments where factor = 0, can we drop them ?
print("Total number of recipe-sub recipe assignments where factor is 0")
len(rec_using_let_srec.loc[rec_using_let_srec['Factor'] == 0])

Total number of recipe-sub recipe assignments where factor is 0


99

In [170]:
# drop rows that have factor = 0 for recipe-subrecipe combination
rec_using_let_srec = rec_using_let_srec[rec_using_let_srec['Factor'] != 0]
len(rec_using_let_srec)

2379

### rec_using_let_srec can have duplicate entries cause a single recipe can use more than 1 sub-recipe containing lettuce

In [171]:
duplicateRecipes = rec_using_let_srec[rec_using_let_srec.duplicated('RecipeId')]
len(duplicateRecipes)


317

In [172]:
b = rec_using_let_srec['RecipeId'].unique().tolist()
print("Total number of unique recipes that use sub-recipes containing lettuce")
len(b)

Total number of unique recipes that use sub-recipes containing lettuce


2062

In [173]:
# cross checking of all the sub-recipes containing lettuce are found in the list of recipes using sub-recipe with letuce
a = rec_using_let_srec['SubRecipeId'].unique().tolist()
print("Total number of sub-recipes containing lettuce")
len(a)

Total number of sub-recipes containing lettuce


6

In [5]:
#rec_using_let_srec

## Recipe-id <span style="color: red;">CHECKER</span>  #59

#### check what subrecipes(with lettuce) this recipe is using

In [175]:
rec_using_let_srec.loc[rec_using_let_srec['RecipeId'] == 4334]

,RecipeId,SubRecipeId,Factor
7655,4334,51,1


#### check what ingredients this recipe is using

In [176]:
isin

,RecipeId,IngredientId,Quantity
87,59,2,4
88,59,7,2
89,59,13,2


#### check what ingredients the sub-recipe is using 

In [177]:
sub_recp_ing.loc[sub_recp_ing['SubRecipeId'] == 10]

,SubRecipeId,IngredientId,Quantity
20,10,17,0
21,10,18,1
22,10,19,1
23,10,20,1
24,10,25,1
25,10,26,1
26,10,27,2
27,10,28,2
28,10,90,1
29,10,120,1


In [178]:
sub_recp_ing.loc[sub_recp_ing['SubRecipeId'] == 5]

,SubRecipeId,IngredientId,Quantity
15,5,27,4
16,5,85,1
17,5,109,1
18,5,153,1


<b>Useful variables</b>
1. rec_using_let_srec
2. lettuce_recipes
3. lettuce_sub_recipes

<b>Useful Lists</b>   
1. letRecipes  
2. letSubRecipes  

### All recipes from the recipe table using lettuce

In [179]:
lettuce_recipes.head(30)

,RecipeId,IngredientId,Quantity
1780,959,27,12
1791,960,27,24
1969,1044,27,10
2295,1220,27,5
3410,1778,27,5
3469,1815,27,1
3475,1816,27,1
3480,1817,27,1
3486,1818,27,1
3491,1819,27,1


### Recipes that are in lettuce recipes (using lettuce directly) are also present in rec_using_let_srec (using lettuce indirectly through sub-recipes) - 
### directly and indirectly 16 recipes & 14 recipes directly ONLY

In [180]:
# find recipes that are present in both the dataframes
lettuce_recipes['RecipeId'].isin(rec_using_let_srec['RecipeId']).value_counts()

True     16
False    14
Name: RecipeId, dtype: int64

### convert lettuce_recipes into a list

In [181]:
# list of recipes using lettuce
letRecipes = lettuce_recipes['RecipeId'].tolist() # lettuce_recipes list


In [182]:
rec_using_let_srec.head()

,RecipeId,SubRecipeId,Factor
2,2,10,1
6,6,10,1
9,7,10,1
14,8,10,1
16,9,5,1


In [183]:
sub_recp_ing.head()

,SubRecipeId,IngredientId,Quantity
0,1,1,1
1,1,135,2
2,1,138,1
3,1,146,1
4,1,157,1


### DIRECT AND INDIRECT RECIPES

NOTE : you got to add both direct and indirect usage 

In [184]:
rec_dir_indir = rec_using_let_srec[rec_using_let_srec['RecipeId'].isin(letRecipes)]
rec_dir_indir


,RecipeId,SubRecipeId,Factor
7653,4333,51,1
7655,4334,51,1
7780,4394,51,1
7782,4396,51,1
7784,4397,51,1
7786,4398,51,1
7788,4399,51,1
7790,4400,51,1
7792,4401,51,1
7794,4402,51,1


## list recipes that is using lettuce directly and indirectly 

In [185]:
DirIndirRec = rec_dir_indir['RecipeId'].tolist()
DirIndirRec

[4333,
 4334,
 4394,
 4396,
 4397,
 4398,
 4399,
 4400,
 4401,
 4402,
 4403,
 4404,
 4591,
 7032,
 7034,
 7036]

### 1. recipes that is using lettuce directly ONLY


In [186]:
DirReconly = list(set(letRecipes) - set(DirIndirRec))
DirReconly

[960,
 3553,
 1220,
 1831,
 1832,
 1778,
 1044,
 1815,
 1816,
 1817,
 1818,
 1819,
 2237,
 959]

In [187]:
df1 = lettuce_recipes[lettuce_recipes['RecipeId'].isin(DirReconly)]
df1

,RecipeId,IngredientId,Quantity
1780,959,27,12
1791,960,27,24
1969,1044,27,10
2295,1220,27,5
3410,1778,27,5
3469,1815,27,1
3475,1816,27,1
3480,1817,27,1
3486,1818,27,1
3491,1819,27,1


### <span style="color: red;">CHECKER</span> : Just checking if direct lettuce using recipes are in recipes subrecipes with lettuce

In [188]:
rec_using_let_srec.loc[rec_using_let_srec['RecipeId'] == 1831]

,RecipeId,SubRecipeId,Factor


### 2. recipes that is using lettuce BOTH directly AND indirectly via sub-recipes

#### recipes that is using lettuce directly - todo df2 + df3

#### 2.a DIRECT PART

In [189]:
# direct part list

# SUBTRACTING THE DIRECTINDIRECT FROM ALL RECIPES USING LETTUCE

df2 = lettuce_recipes[lettuce_recipes['RecipeId'].isin(DirIndirRec)]
df2

,RecipeId,IngredientId,Quantity
8248,4333,27,12
8259,4334,27,24
8396,4394,27,12
8407,4396,27,12
8417,4397,27,12
8427,4398,27,12
8437,4399,27,12
8448,4400,27,12
8459,4401,27,12
8470,4402,27,24


In [190]:
rec_with_subrec = df2['RecipeId'].tolist()
rec_with_subrec

[4333,
 4334,
 4394,
 4396,
 4397,
 4398,
 4399,
 4400,
 4401,
 4402,
 4403,
 4404,
 4591,
 7032,
 7034,
 7036]

#### 2.b INDIRECT PART df3 also has the same number of recipe list as the direct part

In [ ]:
df3 = recp_sub_recp_ass[recp_sub_recp_ass['RecipeId'].isin(rec_with_subrec)]


In [192]:
letSubRecipes

[5, 10, 49, 51, 58, 73]

In [193]:
 # indirect part list
df3 = df3[df3['SubRecipeId'].isin(letSubRecipes)]
df3

,RecipeId,SubRecipeId,Factor
7653,4333,51,1
7655,4334,51,1
7780,4394,51,1
7782,4396,51,1
7784,4397,51,1
7786,4398,51,1
7788,4399,51,1
7790,4400,51,1
7792,4401,51,1
7794,4402,51,1


In [194]:
rec_using_let_srec.shape

(2379, 3)

## 3. recipes that use lettuce ONLY through sub-recipes

#### In the next step we are removing all the direct-indirect combination of recipes to get list of recipes that purely use subrecipes containing lettuce

In [195]:
df4 = rec_using_let_srec[~rec_using_let_srec['RecipeId'].isin(rec_with_subrec)]
df4.shape

(2363, 3)

In [196]:
df4.loc[df4['RecipeId'] == 7036]

,RecipeId,SubRecipeId,Factor


In [197]:
factors = df4['Factor'].unique().tolist()
factors

[1, 3, 5, 7, 11]

In [198]:
df4 = df4[df4['Factor'] != 0]

In [199]:
df4.shape

(2363, 3)

In [200]:
d = df4['RecipeId'].unique().tolist()
len(d)

2046

In [ ]:
dup = df4[df4.duplicated('RecipeId')]


## Calculate the amount of lettuce used by these sub-recipes

In [202]:
lettuce_sub_recipes

,SubRecipeId,IngredientId,Quantity
15,5,27,4
26,10,27,2
239,49,27,2
253,51,27,2
279,58,27,4
329,73,27,2


1. create a dictionary of {subrecipeid, qty}
2. in df3 lookup subrecipe id in the dictionary and multiply factor * qty into a [new column]

### Make a dictionary of subrecipe - lettuce quantity 

In [203]:
subrecipe_letqty_dict = lettuce_sub_recipes.set_index('SubRecipeId').to_dict()['Quantity']
subrecipe_letqty_dict


{5: 4, 10: 2, 49: 2, 51: 2, 58: 4, 73: 2}

### converting factor to quantity in df3 

In [204]:
Quantity = []
for index, row in df3.iterrows():
    sub_rec_id = row['SubRecipeId']
    factor = row['Factor']
    value_at_index = subrecipe_letqty_dict.get(sub_rec_id)
    Quantity.append(factor * value_at_index)


In [205]:
df3["Quantity"] = Quantity 

<ipython-input-205-36fedae255a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["Quantity"] = Quantity


In [206]:
df3

,RecipeId,SubRecipeId,Factor,Quantity
7653,4333,51,1,2
7655,4334,51,1,2
7780,4394,51,1,2
7782,4396,51,1,2
7784,4397,51,1,2
7786,4398,51,1,2
7788,4399,51,1,2
7790,4400,51,1,2
7792,4401,51,1,2
7794,4402,51,1,2


In [207]:
df2 

,RecipeId,IngredientId,Quantity
8248,4333,27,12
8259,4334,27,24
8396,4394,27,12
8407,4396,27,12
8417,4397,27,12
8427,4398,27,12
8437,4399,27,12
8448,4400,27,12
8459,4401,27,12
8470,4402,27,24


### summing df2 and df3 (these 2 are same recipes that use lettuce directly and also indirectly via subrecipes)

In [208]:

df2_df3 = pd.merge(df2,df3, on='RecipeId', how='outer')

In [209]:
df2_df3["Quantity"] = df2_df3["Quantity_x"] + df2_df3["Quantity_y"]

In [210]:
df2_df3

,RecipeId,IngredientId,Quantity_x,SubRecipeId,Factor,Quantity_y,Quantity
0,4333,27,12,51,1,2,14
1,4334,27,24,51,1,2,26
2,4394,27,12,51,1,2,14
3,4396,27,12,51,1,2,14
4,4397,27,12,51,1,2,14
5,4398,27,12,51,1,2,14
6,4399,27,12,51,1,2,14
7,4400,27,12,51,1,2,14
8,4401,27,12,51,1,2,14
9,4402,27,24,51,1,2,26


In [211]:
df2_df3 = df2_df3.drop(["Quantity_x", "Quantity_y", "Factor"], axis = 1, inplace=False)

In [212]:
df2_df3.shape


(16, 4)

In [213]:
df2_df3

,RecipeId,IngredientId,SubRecipeId,Quantity
0,4333,27,51,14
1,4334,27,51,26
2,4394,27,51,14
3,4396,27,51,14
4,4397,27,51,14
5,4398,27,51,14
6,4399,27,51,14
7,4400,27,51,14
8,4401,27,51,14
9,4402,27,51,26


In [214]:
df1.reset_index(drop=True, inplace=True)
df1.index += 1
df1

,RecipeId,IngredientId,Quantity
1,959,27,12
2,960,27,24
3,1044,27,10
4,1220,27,5
5,1778,27,5
6,1815,27,1
7,1816,27,1
8,1817,27,1
9,1818,27,1
10,1819,27,1


In [7]:
#df4

### <span style="color: red;">CHECKER </span> checking if we removed these DirIndirRec combination recipes from the df4 (df4 is recipes using only subrecipes with lettuce)

In [216]:
df4[df4['RecipeId'].isin(DirIndirRec)]

,RecipeId,SubRecipeId,Factor


In [217]:
df4[df4['RecipeId'].isin(DirReconly)]

,RecipeId,SubRecipeId,Factor


In [218]:
Quant = []
for index, row in df4.iterrows():
    sub_rec_id = row['SubRecipeId']
    factor = row['Factor']
    value_at_index = subrecipe_letqty_dict.get(sub_rec_id)
    Quant.append(factor * value_at_index)
df4["Quantity"] = Quant

In [8]:
#df4


{5: 4, 10: 2, 49: 2, 50: 0, 51: 2, 58: 4, 73: 2}

In [9]:
#df4.loc[df4['SubRecipeId'] == 5]

####   grouping the recipe ids and totalling their lettuce quantity in df4, since df4 has duplicate recipe ids

In [124]:
df4 = df4.drop(["Factor"], axis = 1, inplace=False)


In [ ]:

df4.reset_index(drop=True, inplace=True)

In [126]:
df4_grouped = df4.groupby(by="RecipeId")["Quantity"].sum().reset_index()

In [127]:
df4_grouped

,RecipeId,Quantity
0,2,2
1,6,2
2,7,2
3,8,2
4,9,6
5,10,2
6,11,2
7,12,2
8,13,2
9,14,6


In [128]:
df4_grouped.shape

(2046, 2)

###  <span style="color: red;">Final Recipe-Quantity Table formation </span> combine df1 (14 recipes), df2_df3 (16 recipes), df4_grouped (2046 recipes) with columns RecipeId, LettuceQuantity

In [220]:
14 + 16  +2046

2076

#### 1. adding df1 to data

In [221]:
data = df1
data = data.drop('IngredientId', axis=1)
data

,RecipeId,Quantity
1,959,12
2,960,24
3,1044,10
4,1220,5
5,1778,5
6,1815,1
7,1816,1
8,1817,1
9,1818,1
10,1819,1


#### 2. adding df4_grouped to data

In [222]:
data = pd.concat([data, df4_grouped], axis=0, ignore_index=True)

In [ ]:
data.shape

#### 3. adding df2_df3 to data

In [224]:
2060 + 16

2076

In [225]:
data = pd.concat([data, df2_df3], axis=0, ignore_index=True)

In [ ]:
data

In [227]:
data = data.drop(['IngredientId', 'SubRecipeId'], axis=1)

In [228]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data

## RecipeId and quantity of lettuce used dataframe: "<span style="color: red;">data</span>" (A total of 2076 recipes)

In [230]:
zero_quantity_lettuce = data.loc[data['Quantity'] == 0]
len(zero_quantity_lettuce)

0

In [231]:
data = data[data['Quantity'] != 0]

In [232]:
data.shape

(2076, 2)

In [233]:
data.isnull().values.any()

False

### *************

## menu_item (Menu card)

In [246]:
menu_items = pd.read_csv(placeholder + 'menu_items.csv')
#ingredients['IngredientName'] = ingredients['IngredientName'].str.strip() #stripping whitespace
menu_items.head()

,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
0,FtL/Ham,Ham FtLong,111000004,1,6
1,Six/Ham,Ham 6 inch,121000004,2,7
2,FfB/Ham,Ham FtFbd,112000004,3,2
3,fBd/Ham,Ham FlatBd,122000004,4,8
4,Sld/Ham,Ham Salad,131000004,5,9


In [247]:
menu_items.shape

(8005, 5)

In [248]:
#checking if there are any null values
menu_items.isnull().values.any()

False

## menu_items have duplicate entries  <span style="color: red;">DO NOT USE MenuItemId variable alone, use PLU which is a unique identifier of a menu item I think</span>

In [ ]:
dupli = menu_items[menu_items.duplicated('RecipeId')]
dupli

## Number of recipes that have more than one	MenuItemId - 1083 recipes

In [258]:
menu_items.loc[menu_items['RecipeId'] == 3781]


,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
4103,ad6/B.M.T.,B.M.T. Add6in,175000008,4121,3781
6454,ad6/B.M.T.,B.M.T. Add6in,175000008,7471,3781
6658,ad6/B.M.T.,B.M.T. Add6in,175000008,7879,3781


#### Checking if every menuitem in the menu card has a unique recipe id, in other words, check 1-to-1 matching of MenuItemId and RecipeId. 
#### RESULT: They are 1-to-1

In [243]:
menu_items.head(5)

,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
0,FtL/Ham,Ham FtLong,111000004,1,6
1,Six/Ham,Ham 6 inch,121000004,2,7
2,FfB/Ham,Ham FtFbd,112000004,3,2
3,fBd/Ham,Ham FlatBd,122000004,4,8
4,Sld/Ham,Ham Salad,131000004,5,9


In [275]:
#menu_group = menu_items.groupby(['RecipeId', 'PLU']).size().reset_index().rename(columns={0:'count'})
# sorted_menu_group = menu_group.sort_values(by=['RecipeId'], ascending=True)

### Prepare menu card with only lettuce based menu. Pick only items in the menu_item or recipeIds that have lettuce in them (use "data" and menu_items dataframe for this)

In [572]:
data.shape

(2076, 2)

In [276]:
menu_items.shape

(8005, 5)

In [277]:
menu_items.head()

,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
0,FtL/Ham,Ham FtLong,111000004,1,6
1,Six/Ham,Ham 6 inch,121000004,2,7
2,FfB/Ham,Ham FtFbd,112000004,3,2
3,fBd/Ham,Ham FlatBd,122000004,4,8
4,Sld/Ham,Ham Salad,131000004,5,9


In [278]:
data.head()

,RecipeId,Quantity
0,959,12
1,960,24
2,1044,10
3,1220,5
4,1778,5


#### The total number of recipe ids in lettuce_menu  should be <= 2076 (which are lettuce based recipes)

In [279]:
#lettuce_menu = pd.merge(data,menu_items[['RecipeId','PLU','MenuItemId']],on='RecipeId', how='left')
lettuce_menu = pd.merge(menu_items[['RecipeId','PLU','MenuItemId']], data, on='RecipeId', how='left')

In [ ]:
lettuce_menu

In [292]:
# checking how many Nan values each of the columns of lettuce_menu have
lettuce_menu.isna().sum()

RecipeId         0
PLU              0
MenuItemId       0
Quantity      5877
dtype: int64

### if quantity is Nan in lettuce_menu, it implies its not a lettuec based menu item and therefore be dropped from the menu card

In [295]:
lettuce_menu = lettuce_menu[lettuce_menu['Quantity'].notna()]
lettuce_menu.shape

(2128, 4)

In [296]:
e = lettuce_menu['RecipeId'].unique().tolist()
len(e)

1982

In [ ]:
duplic = lettuce_menu[lettuce_menu.duplicated('RecipeId')]
duplic

In [298]:
lettuce_menu.loc[lettuce_menu['RecipeId'] == 320]

,RecipeId,PLU,MenuItemId,Quantity
294,320,186000048,295,2.0
1047,320,186000048,1053,2.0


In [299]:
lettuce_menu.shape

(2128, 4)

In [300]:
lettuce_menu['MenuItemId'].isnull().values.any()


False

### <span style="color: red;"> Voila!!! </span> the following number is less (lettuce based items on menu card/menu_items) than the total number of lettuce based recipes the company has

In [301]:
unique_recipeId = lettuce_menu['RecipeId'].unique().tolist()
len(unique_recipeId)

1982

Important tables  

1. lettuce menu - has RecipeId, PLU, MenuItemID, Quuantity (of lettuce) - All lettuce based recipes on the menu card 
2. data  - has RecipeId, Quantity - All the lettuce based recipes of SubWay

##### From code block above, we can be sure that lettuce_menu dataframe has the complete list of recipes on the menu card, that use lettuce with their quantity of usage

### *****************************

### Transactions

In [303]:
pos = pd.read_csv(placeholder + 'pos_ordersale.csv')
pos.head()

,MD5KEY_ORDERSALE,ChangeReceived,OrderNumber,TaxInclusiveAmount,TaxAmount,MealLocation,TransactionId,StoreNumber,date
0,00BCB738A1F814DB960C25012E9913DF,0.00,67,0,0.56,0,67,46673,15-03-05
1,03438C257BC2DD0037F35EC8BFC0C8E2,3.46,39,0,0.54,0,39,12631,15-03-05
2,04AEF56CF3130AC1EFAD41C470F5245D,0.00,81,0,0.69,0,79,46673,15-03-05
3,04BBFAD4E73C30EE2EC3B4E52509637C,0.00,159,0,1.16,0,159,12631,15-03-05
4,061AE382DC127057E8ADBBF869CFB361,0.00,51,0,0.00,0,51,12631,15-03-05


### TODO

1.Drop all stores beside 46673  
2.Group transactions date wise  
3.Look up menuitems to see what PLU was bought for a particular order_sale key (we only interested in PLUs that have lettuce)  
4. 

### Select only transactions that belong to store 46673

In [304]:
pos_46673 = pos[pos['StoreNumber'] == 46673]
pos_46673.shape

(7197, 9)

#### Drop unecessary columns from the pos_46673 table

In [305]:
pos_46673 = pos_46673.drop(["ChangeReceived", "OrderNumber", "TaxInclusiveAmount", "TaxAmount","MealLocation", "TransactionId","StoreNumber"], axis = 1, inplace=False)


### Total number of transactions (7197) happened in store # 46673 in between 3 months (15th Mar - 15th June)

In [ ]:
pos_46673.reset_index(drop=True)

### menuitem

In [312]:
menuitem = pd.read_csv(placeholder + 'menuitem.csv')
menuitem.head()

,MD5KEY_MENUITEM,MD5KEY_ORDERSALE,CategoryDescription,DepartmentDescription,Description,StoreNumber,TaxInclusiveAmount,TaxAmount,AdjustedPrice,DiscountAmount,Price,Quantity,PLU,Id,date
0,001E0224E4EDA63D6FED96D59E0C1E76,072D3B76BE58E4553A373AF3206D2CA5,Footlong,Footlong Sub,Chickn Strips FtLong,46673,0,0.6975,7.75,0.00,7.75,1,111000041,326,15-03-05
1,003E4E20196B2F5BDD33CE06D2943EF4,14811D4F3458058007B0B6908C2DA393,Fountain,Fountain Drink 21 oz.,21oz Fountain Drink,46673,0,0.1530,1.70,0.00,1.70,1,212000126,280,15-03-05
2,012A55E5551C95A559A7B074AE85117E,22FA6C39B7A15ABB6EB568E66A6B30E1,6 Inch,6 Inch Sub,B.M.T. 6 inch,12631,0,0.3230,3.64,0.61,4.25,1,121000008,8,15-03-05
3,01B07F1C6DC6945796571FC78735DA16,915B69612845BBD974C39AC105F5CA8C,6 Inch,6 Inch Sub,Veggie Delite 6 inch,12631,0,0.3328,3.75,0.00,3.75,1,121000001,91,15-03-05
4,02DE19974DE5304BF243A1C2EC639495,CD6386C802450EDCF13A3D112D1F86EC,Add On,Add On 6in Portion,Avocado Add6in,12631,0,0.0443,0.50,0.00,0.50,1,175000062,564,15-03-05


#### total number of menu items ordered in store #46673 - 17111 items of which some are not lettuce based. We are only interested in lettuce based items

In [313]:
menuitem_46673 = menuitem[menuitem['StoreNumber'] == 46673]
menuitem_46673.shape

(17111, 15)

#### Drop unecessary columns from the menuitem_46673 table


In [314]:
menuitem_46673 = menuitem_46673.drop(["DepartmentDescription", "CategoryDescription", "Description", "StoreNumber", "TaxInclusiveAmount","TaxAmount", "AdjustedPrice","DiscountAmount","Price"], axis = 1, inplace=False)

#### renaming column Id -> MenuItemId for easy operation between the menuitem_46673 and lettuce_menu tables

In [315]:
menuitem_46673.rename(columns = {'Id':'MenuItemId'}, inplace = True)

In [316]:
menuitem_46673.head()

,MD5KEY_MENUITEM,MD5KEY_ORDERSALE,Quantity,PLU,MenuItemId,date
0,001E0224E4EDA63D6FED96D59E0C1E76,072D3B76BE58E4553A373AF3206D2CA5,1,111000041,326,15-03-05
1,003E4E20196B2F5BDD33CE06D2943EF4,14811D4F3458058007B0B6908C2DA393,1,212000126,280,15-03-05
5,03F5700F4CB7F5388F217E1532F9A0C3,2B6FD9058A4A28E5E298C95D3C615B27,1,121000012,13,15-03-05
11,0A213180709536CABCEA0A73658D1C32,5A3C75477697E093A0B8B81A43A60424,1,212000126,280,15-03-05
13,0C3A6213CD81B91180D6876CDFAE7A91,1E739D78994A32D295D9C6B6F2D76025,1,111000012,12,15-03-05


### <span style="color: red;"> CHECKER </span> checking entry from menuitem in lettuce_menu

In [658]:
lettuce_menu.loc[(lettuce_menu['PLU'] == 121000012) & (lettuce_menu['MenuItemId'] == 13)]

,RecipeId,Quantity,PLU,MenuItemId
25,12,2,121000012.0,13.0


#### <span style="color: red;">NOTE : RecipeId and PLU are 1-to-1 and uniquely identify a single recipe</span>

### Keep only items on menuitem table are actually lettuce based - do the matching based on PLU and MenuItemId between dataframes lettuce_menu and menuitem_46673 

In [318]:
new_df =pd.merge(menuitem_46673, lettuce_menu,  how='left', left_on=['PLU','MenuItemId'], right_on = ['PLU','MenuItemId'])


In [319]:
new_df.shape

(17111, 8)

In [1]:
#new_df

### if quantity is Nan in RecipeId, it implies the ordered item is not a lettuce based menu item and therefore be dropped from menuitem table that holds all items that were ordered in store 46673

In [321]:
# checking how many Nan values each of the columns of lettuce_menu have
new_df.isna().sum()

MD5KEY_MENUITEM         0
MD5KEY_ORDERSALE        0
Quantity_x              0
PLU                     0
MenuItemId              0
date                    0
RecipeId            10240
Quantity_y          10240
dtype: int64

In [322]:
new_df = new_df[new_df['Quantity_y'].notna()]
new_df.shape

(6871, 8)

In [327]:
# reset the index
new_df.reset_index(drop=True, inplace=True)

In [328]:
new_df.drop(["MD5KEY_MENUITEM"], axis = 1, inplace=True)

<ipython-input-328-60bca2f30d5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(["MD5KEY_MENUITEM"], axis = 1, inplace=True)


In [330]:
new_df.rename(columns = {'Quantity_x':'Amount', 'Quantity_y':'LettuceQuantity'}, inplace = True)

<ipython-input-330-60e1bc16f662>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns = {'Quantity_x':'Amount', 'Quantity_y':'LettuceQuantity'}, inplace = True)


### <span style="color: red;"> Below dataframe is the final one - new_df </span>

#### Next step is the match the order transaction numbers between pos_46673 and new_df, and do group by dates

In [2]:
#new_df

In [332]:
new_df.shape

(6871, 7)

TODO
1. <span style="color: red;"> I think date is of the format YYYY-MM-DD </span>
2. Multiply amount and LettuceQuantity for each row
3. I think I only need to group the rows based on date and sum the lettuce quantity


## Do I really need this pos_46673 ?

In [333]:
pos_46673.head()

,MD5KEY_ORDERSALE,date
0,00BCB738A1F814DB960C25012E9913DF,15-03-05
2,04AEF56CF3130AC1EFAD41C470F5245D,15-03-05
7,072D3B76BE58E4553A373AF3206D2CA5,15-03-05
8,07436936F3929B875D3128106550394F,15-03-05
11,0AAAFC1D4FBFF5ADFF1CD0DD2B154F3B,15-03-05


### <span style="color: red;">  CHECKER </span> check the demand of lettuce per day

In [343]:
Mar6 = new_df.loc[new_df['date'] == '15-03-06']

In [344]:
Mar6.shape


(44, 7)

In [345]:
Mar6

,MD5KEY_ORDERSALE,Amount,PLU,MenuItemId,date,RecipeId,LettuceQuantity
74,0E88F051929D6A4976AB04A044EF0130,1,121000017,49,15-03-06,52.0,2.0
75,9380C87E8D18DD44C1C4B0E80502DAED,1,121000008,8,15-03-06,47.0,2.0
76,A08ED8B0142425E22901363D9D6F52DF,1,121000021,39,15-03-06,104.0,2.0
77,87BF60C859DBE424AD8B5C5CDE295DB9,1,111000054,152,15-03-06,233.0,2.0
78,47727BE7AC49BF08E5C1A66EEE2B704B,1,111000004,1,15-03-06,6.0,2.0
79,F773A17F79C422A0A87407ECF50CE63E,1,121000018,34,15-03-06,67.0,2.0
80,19E1415CA29D2D68FD146A64B8719ECE,2,111000006,74,15-03-06,75.0,2.0
81,B502592073BB884C007BF3DD426F4504,1,111000006,74,15-03-06,75.0,2.0
82,6F18473EC6C794F97BBD51AAD0B2C3AF,1,121000006,75,15-03-06,77.0,2.0
83,ED7577AF484390119F0F55A8D36CAB40,1,121000018,34,15-03-06,67.0,2.0


 ### Get the total lettuce qty per row by multiplying amt * qty

In [349]:
new_df['lettuce_qty_per_row'] = new_df['Amount'] * new_df['LettuceQuantity']

<ipython-input-349-1ad9cd5842f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['lettuce_qty_per_row'] = new_df['Amount'] * new_df['LettuceQuantity']


In [3]:
#new_df

### Group by date to get the demand for lettuce for each day

In [ ]:
series = new_df.groupby('date')['lettuce_qty_per_row'].sum().reset_index()
series

In [352]:
series.rename(columns = {'lettuce_qty_per_row':'lettuceDemand'}, inplace = True)

In [353]:
series.head()

,date,lettuceDemand
0,15-03-05,152.0
1,15-03-06,100.0
2,15-03-07,54.0
3,15-03-08,199.0
4,15-03-09,166.0




## END of Data Transformation

## convert dataframe to csv file

In [354]:
series.to_csv('lettuce_series.csv')